In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !rm -rf saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model/lightning_logs

In [3]:
import pandas as pd
import numpy as np
import torch
import lightning.pytorch as pl
import matplotlib.pyplot as plt
# import plotly
import plotly.express as px
import os
import torch
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from src.modules.lifter_2d_3d.model.linear_model.linear_model import BaselineModel
from src.modules.lifter_2d_3d.dataset.drive_and_act_keypoint_dataset import DriveAndActKeypointDataset
from src.modules.lifter_2d_3d.model.linear_model.lit_linear_model import LitSimpleBaselineLinear
from src.modules.utils.visualization import generate_connection_line, get_sample_from_loader, visualize_pose
from IPython.display import display

pl.seed_everything(1234)

Global seed set to 1234


1234

In [4]:
train_dataset = DriveAndActKeypointDataset(
    # prediction_file="/root/data/processed/synthetic_cabin_bw/A_Pillar_Codriver/keypoint_detection_results/keypoint_detection_train.json",
    # annotation_file="/root/data/processed/synthetic_cabin_bw/A_Pillar_Codriver/annotations/person_keypoints_train.json",
    prediction_file="/root/data/processed/drive_and_act/keypoint_detection_results/keypoint_detection_train.json",
    annotation_file="/root/data/processed/drive_and_act/annotations/person_keypoints_train.json",
    image_width=1280,
    image_height=1024,
    actors=['vp1', 'vp2', 'vp3', 'vp4', 'vp5', 'vp6', 'vp7', 'vp8', 'vp9', 'vp10'],
    exclude_ankle=True,
    exclude_hip=True
)
val_dataset = DriveAndActKeypointDataset(
    # prediction_file="/root/data/processed/synthetic_cabin_bw/A_Pillar_Codriver/keypoint_detection_results/keypoint_detection_val.json",
    # annotation_file="/root/data/processed/synthetic_cabin_bw/A_Pillar_Codriver/annotations/person_keypoints_val.json",
    prediction_file="/root/data/processed/drive_and_act/keypoint_detection_results/keypoint_detection_val.json",
    annotation_file="//root/data/processed/drive_and_act/annotations/person_keypoints_val.json",
    image_width=1280,
    image_height=1024,
    actors=['vp11', 'vp12'],
    exclude_ankle=True,
    exclude_hip=True
)
test_dataset = DriveAndActKeypointDataset(
    # prediction_file="/root/data/processed/synthetic_cabin_bw/A_Pillar_Codriver/keypoint_detection_results/keypoint_detection_test.json",
    # annotation_file="/root/data/processed/synthetic_cabin_bw/A_Pillar_Codriver/annotations/person_keypoints_test.json",
    prediction_file="/root/data/processed/drive_and_act/keypoint_detection_results/keypoint_detection_test.json",
    annotation_file="/root/data/processed/drive_and_act/annotations/person_keypoints_test.json",
    image_width=1280,
    image_height=1024,
    actors=['vp13', 'vp14', 'vp15'],
    exclude_ankle=True,
    exclude_hip=True
)

print(
    'train_dataset', len(train_dataset),
    'val_dataset', len(val_dataset),
    'test_dataset', len(test_dataset)
)
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True, num_workers=24)
val_loader = DataLoader(val_dataset, batch_size=64, drop_last=True, num_workers=24)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=24)

model_checkpoint = ModelCheckpoint(monitor='val_loss',mode='min', save_top_k=1)
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=5)

# ------------
# model
# ------------
lit_model = LitSimpleBaselineLinear(exclude_ankle=True, exclude_hip=True, all_activities=train_dataset.activities)
# ------------
# training
# ------------
# saved_model_path = './saved_lifter_2d_3d_model/synthetic_cabin_bw/A_Pillar_Codriver/prediction/linear_model/'
saved_model_path = './saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model'
if not os.path.exists(saved_model_path):
    os.makedirs(saved_model_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainer = pl.Trainer(
    # max_steps=10,
    max_epochs=200,
    callbacks=[model_checkpoint, early_stopping],
    accelerator=device,
    check_val_every_n_epoch=5,
    default_root_dir=saved_model_path,
    gradient_clip_val=1.0
)
trainer.fit(lit_model, train_loader, val_loader)

skipping problematic image 15783
skipping problematic image 17258
skipping problematic image 17259
skipping problematic image 21271
skipping problematic image 21272
skipping problematic image 21273
skipping problematic image 21274
skipping problematic image 21275
skipping problematic image 21276
skipping problematic image 33527


/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model/lightning_logs

  | Name  | Type          | Params
----------------------------------------
0 | model | BaselineModel | 4.3 M 
----------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.105    Total estimated model params size (MB)


train_dataset 26626 val_dataset 5103 test_dataset 8414


Sanity Checking: 0it [00:00, ?it/s]

check #0
val MPJPE from: 0 batches : 2893.498182296753


...

check #20
training loss from 2080 batches: 42.98719220723097
val MPJPE from: 0 batches : 57.94432759284973


Validation: 0it [00:00, ?it/s]

check #21
training loss from 2080 batches: 42.92638946969349
val MPJPE from: 0 batches : 57.69021064043045


Validation: 0it [00:00, ?it/s]

check #22
training loss from 2080 batches: 42.88366494986873
val MPJPE from: 0 batches : 57.726215571165085


In [5]:
best_checkpoint_path = model_checkpoint.best_model_path
trainer.test(ckpt_path=best_checkpoint_path, dataloaders=test_loader)
results = lit_model.evaluator.get_result()

Restoring states from the checkpoint path at saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model/lightning_logs/version_0/checkpoints/epoch=84-step=35360.ckpt
Loaded model weights from the checkpoint at saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model/lightning_logs/version_0/checkpoints/epoch=84-step=35360.ckpt


Testing: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


MPJPE: 62.34972808374529
PJPE
[ 43.69626806  39.99918325  69.78556192 ...  89.83796079 135.48030462
 183.90687119]
activities_mpjpe:
{'exiting_car': 170.89531486205266, 'opening_laptop': 63.51475101620452, 'closing_door_outside': nan, 'drinking': 58.31714203813158, 'fastening_seat_belt': 78.22075928451402, 'taking_off_jacket': nan, 'closing_bottle': 59.07290063759016, 'using_multimedia_display': 76.76192723121257, 'eating': 54.61526656879297, 'taking_laptop_from_backpack': 80.38728256039973, 'taking_off_sunglasses': 77.82238147125821, 'closing_backpack': nan, 'placing_an_object': 89.3160788081485, 'writing': 65.27003167644445, 'reading_magazine': 59.84465232813644, 'working_on_laptop': 50.88960020077223, 'preparing_food': 57.88966049651875, 'closing_door_inside': 99.60473563520098, 'looking_or_moving_around (e.g. searching)': 99.05439668042725, 'looking_back_right_shoulder': nan, 'putting_on_sunglasses': 75.94823909621174, 'opening_bottle': 67.82618262394126, 'interacting_with_phone': 

/workspace/src/modules/lifter_2d_3d/utils/evaluation.py:55: RuntimeWarning: Mean of empty slice.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           mpjpe           │     62.34972808374529     │
└───────────────────────────┴───────────────────────────┘

In [15]:
np.array(lit_model.evaluator.pjpe).mean(axis=0) * 1000

array([ 52.34725204,  43.23466746,  45.81155874,  11.02201004,
        43.44939469,  50.37476808,  41.41979644,  66.92523146,
        67.29112654,  81.75081595, 103.93291346,  41.80421584,
        37.21552725])

In [6]:
results

(array([ 43.69626806,  39.99918325,  69.78556192, ...,  89.83796079,
        135.48030462, 183.90687119]),
 62.34972808374529,
 {'exiting_car': 170.89531486205266,
  'opening_laptop': 63.51475101620452,
  'closing_door_outside': nan,
  'drinking': 58.31714203813158,
  'fastening_seat_belt': 78.22075928451402,
  'taking_off_jacket': nan,
  'closing_bottle': 59.07290063759016,
  'using_multimedia_display': 76.76192723121257,
  'eating': 54.61526656879297,
  'taking_laptop_from_backpack': 80.38728256039973,
  'taking_off_sunglasses': 77.82238147125821,
  'closing_backpack': nan,
  'placing_an_object': 89.3160788081485,
  'writing': 65.27003167644445,
  'reading_magazine': 59.84465232813644,
  'working_on_laptop': 50.88960020077223,
  'preparing_food': 57.88966049651875,
  'closing_door_inside': 99.60473563520098,
  'looking_or_moving_around (e.g. searching)': 99.05439668042725,
  'looking_back_right_shoulder': nan,
  'putting_on_sunglasses': 75.94823909621174,
  'opening_bottle': 67.82618

In [ ]:
import json
predictions = {}
with open("/root/data/processed/drive_and_act/keypoint_detection_results/keypoint_detection_train.json") as f:
    data = json.loads(f.read())
    for item in data:
        predictions[item['image_id']] = item

In [ ]:
# best_checkpoint_path = model_checkpoint.best_model_path
# trainer.test(ckpt_path=best_checkpoint_path, dataloaders=test_loader)

In [ ]:
sample = get_sample_from_loader(val_loader)

In [ ]:
valid_keypoints = (sample[1].sum(axis=1) != 0)

In [ ]:
results = generate_connection_line(sample[1],
        np.argwhere(valid_keypoints).reshape(-1))
pose_df = pd.DataFrame(results)
visualize_pose(pose_df)

In [ ]:
model = trainer.model.to(device)
model.eval()
estimated_pose = model(torch.flatten(torch.tensor(sample[0])).unsqueeze(0).float().to(device), 0)
estimated_pose_df = pd.DataFrame(
    generate_connection_line(
        estimated_pose[0].cpu().reshape([-1, 3]).detach().numpy(),
        # np.argwhere(valid_keypoints).reshape(-1)
    )
)
visualize_pose(estimated_pose_df)